In [1]:
import sys
import os

# modify syspath directory to sys.path

package_path = os.path.abspath(os.path.join('..','core'))
if package_path not in sys.path:
    sys.path.append(package_path)

package_path = os.path.abspath(os.path.join('..','utils'))
if package_path not in sys.path:
    sys.path.append(package_path)

package_path = os.path.abspath(os.path.join('..'))
if package_path not in sys.path:
    sys.path.append(package_path)

import numpy as np
import qutip as qt
import math
import matplotlib.pyplot as plt

from IPython.display import HTML # both needed to disply gifs
from PIL import Image

# Import modules from the package
from core import create_hamiltonian as ch
from core import create_state as cs
from core import create_coherent_state
from core import time_evo
from core import time_evo_new
from core import time_evo_from_state
from core import load_param
from core import load_H_list
from core import load_result
from core import load_tlist
from core import operators
from core.schmidt_solve import compute_schmidt_states_all_time
from core.eigen_ener_states import eigen_ener_states as eig
from core import annihilation_operator
from utils.entropy import compute_VN, plot_VN, compute_VN_time
from utils import collapse_gif
from utils.collapse_gif import collapse_gif
from utils.display_gif import display_gif
from utils.distribution_gif import gif_distribution_eig_total
from utils.distribution_gif import prob_gif
from utils.distribution_gif import gif_distrib_H_s_int
from utils.distribution_gif import gif_distrib_system_first_eig
from utils.plot import plot_basic
from utils.plot import plot_with_binning
from utils.basic_functions import state_distribution_eig_tot
from utils.basic_functions import random_dephasing
from utils.basic_functions import get_state_probabilities
from utils.basic_functions import position
from utils.basic_functions import copy_cat
from utils.basic_functions import plot_einselection
from utils.overlap import overlap
from utils.overlap import gif_schmidt_overlap
from utils.interference import interference_plot
from utils.expectations import exp_val_time
from utils.plot_hamiltonians import plot_hamiltonians

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams['axes.titlesize'] = 16             # Title font size
plt.rcParams['axes.titleweight'] = 'bold'       # Title font weight
plt.rcParams['axes.labelsize'] = 14             # X and Y label font size
plt.rcParams['axes.labelweight'] = 'bold'       # X and Y label font weight
#plt.rcParams['axes.grid'] = True                # Enable grid
#plt.rcParams['grid.alpha'] = 0.7                # Grid transparency
#plt.rcParams['grid.linestyle'] = '--'           # Grid line style
#plt.rcParams['grid.color'] = 'gray'             # Grid color

# Tick settings
plt.rcParams['xtick.labelsize'] = 12            # X tick label size
plt.rcParams['ytick.labelsize'] = 12            # Y tick label size
plt.rcParams['xtick.direction'] = 'in'          # X tick direction
plt.rcParams['ytick.direction'] = 'in'          # Y tick direction
plt.rcParams['xtick.major.size'] = 6            # X major tick size
plt.rcParams['ytick.major.size'] = 6            # Y major tick size

# Legend settings
plt.rcParams['legend.fontsize'] = 12            # Legend font size
plt.rcParams['legend.frameon'] = True           # Enable legend frame
plt.rcParams['legend.framealpha'] = 0.9         # Legend frame transparency
plt.rcParams['legend.loc'] = 'best'             # Legend location

# Line and marker settings
plt.rcParams['lines.linewidth'] = 2             # Line width
plt.rcParams['lines.markersize'] = 6            # Marker size

custom_colors = ['#1c4587', '#e6194B', '#2ca02c', '#d62728', '#9467bd', 
                 '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

plt.rcParams['axes.prop_cycle'] = plt.cycler(color=custom_colors)




The goal of this series of jupyter notebook is to be education on decoherence usin the AC model. So the first one should not focuse on the ACL model, but just introduce decoherence itself. Needs a little bit of preliminary knowledge.

RESSOURCES I AM BASING IT ALL ON. 

Note that these notebooks are based on : Lecture notes on quantum decoherence:Jim Al-Khalili, a bunch of Andreas paper.
Sean Carroll's paper for the basi decoherence example.

PLAN FOR THIS NOTEBOOK:
first introduce quantym entanglement
Need a super simple theoretical toy model of deochernece from which we can calculate a few basic qtt. ie, decoherence time scale
Justification for interest. Coudl do something related to the double slit experiment?
Environment causing entanglement for some states, as a selection of pointer states
ZUrek’s commutativity criterion.
Zurek's random walk and convergence to 0.
2 related problems tackeld by decoherence: Preffered basis, non-visibility of quantum interference (ie, prefe basis and classicality.) what about the unicity of measurement results? i feel like it somehow solves it.


PLAN FOR NEXT NOTEBOOKS:
Can I see Zureks' commutativity criterion in the ACL?
ACL model use to talk about decoherence more.
See if you can modify it to study more usual models of decoherence.
see if you can also go into quatum darwinism.
Multiartide version of what I did?
Idea of locality/decoherence somehow being linked. The interactions are local, but the states can be non-local (ie entanglement). And decoherence is what makes us not see that at our scale. what are all the hints for this link?
Decoherence vs dissipation
Decoherence vs classical noise.



There are many things that we can point to and say, look, thats decoherence

LITTLE TRUTHS I WANT TO INCLUDE

Decoherence solves the measurement problem as well as helps us understand the behavior of open (and not so open) quantum systems
Talk about the einselectyion problem and its solution via pointer states.
Decoherence does not destroy superpositions, it simply extends them to include the environment. So, while decoherence is something that happens to the system of interest, what is really going on is that the environment is encoding, via quantum correlations, information about the system.
decoherence leads to the decay of the off diagonal terms in a basis defined by your system environment interactions. it results from your environment states becoming correlated with the states of your system (i.e. effectively measuring your system)
Basically, the fact that the density matrix diagonalizes in the pointer basis and stay diagonal ensures that we have classicaity in this basis. 
Decoherence as a time asymmetric process.
It causes irreversible loss of coherence from the quantum system 2. It limits/selects a small number of possible observables

For the lengh of this notebook, we consider a universe made of 2 subsystems, $S$ and $E$ the system adn the environment.

Decoherence, in addition to being important to understand for quantum computing provides a partial solution to 3 problems in the foundations of physics.

<h1>What is a quantum measurement?


For our purpous a measurement is an interaction between 2 quantum systems. One large and one small. For it to be an interesting measurement, the small system there must be an increase of entropy as the 2 systems interact. I will sometimes refer to the environment as a measuring device, especially when I want to choose the state it measures.

Considering a system in state $|\psi\rangle=\frac{1}{\sqrt{2}}(|0\rangle+|1\rangle)$. We also consider a measuring device in state $|READY\rangle$, as it is ready to measure the system. It is such that feeding it state $|0\rangle$ makes it evolve to 

Not here that we where specifying by hand how the measuring device evolves depending on the state of the system, as well as supposing a non-evolving system. But we can also work with an actual hamiltonian and do the evolution that does the measurement for us. This is already touchingr on a fundamenmtal aspect of decohernece as a solution to fondamental problems. Its all unitary QM. No special evolution for the MWs.

<h1>The prefered basis problem

In a basis $\{|i\rangle\}$ the state of the systsem is $|\psi\rangle=\sum_ia_i|i\rangle$. But the Hilbert space of the system does not have any fundamental structure making basis $\{|i\rangle\}$ preferable over some other basis $\{|i'\rangle\}$. And that basis could even be one such that $|\psi\rangle=\sum_ia_i|i\rangle=|0'\rangle$, that is a basis in which we loose the superposition. If that is the case then a lot of quantum mechanics seems meaningless, why not consider the double slit experiment in the basis where the photons are not in a superposition? Does the computational advantage of QM disapeare if we look at our state in another basis? 

<h1>Where does classicality come from?

<h1>The unicity of measurement outcomes

Considering the Stern Gerlach experiement to understand 